In [ ]:
import numpy as np #needed for matrix algebra
import pandas as pd #for data-frame dataframes are basically 2D arrays
from matplotlib import pyplot as plt #for plotting the images

data = pd.read_csv('/input/digit-recognizer/train.csv') #take the csv data and create a dataframe

In [ ]:
data = np.array(data)  #convert it into a numpy array
m, n = data.shape #m = rows n = columns
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T #take first 1000 rows and transpose them
Y_dev = data_dev[0] #first row of the transposed one is all the labels
X_dev = data_dev[1:n] #rest of the rows is the pixel
X_dev = X_dev / 255. #normalizing the pixel values using float division

data_train = data[1000:m].T #take rest of the rows m here is the total number of rows of the original dataset and transpose them
Y_train = data_train[0] #take the labels row
X_train = data_train[1:n] #take the rest of the rows
X_train = X_train / 255.  #normalizing the pixel values using float division
_,m_train = X_train.shape #aftar the transpose rows are pixel no and columns are no of examples so that is what m_train will store as shape returns rows, colum

In [ ]:
print(m_train)
print(m,n) #42000 examples each row is a training example

In [ ]:
Y_train

In [ ]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5 # the values generated are from [0,1) and this makes it [-0.5,0.5] also we do W*X so W being 10 x 784 and X being 784 by one it becomes 10x 1
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0) #self explanatory

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z)) #our Z is 10x1 so what it does is takes e^z for all elements of Z and divides it with sum of all e^z so we get softmaxed matrix
    return A

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1 #dot is matrix multiplication W1 is 10x784 and X is 784x1 so Z1 is 10x1
    A1 = ReLU(Z1) #A1 is 10x1
    Z2 = W2.dot(A1) + b2 #W2 is 10X10 and A1 is 10x 1 so Z2 is 10 x 1
    A2 = softmax(Z2) #A2 is 10x 1
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1)) #size is total number of element say shape gives us m,n then size gives us m*n  and other gives us maximum element+1 ie 9+1 so the shape is (number of labels,10)
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [ ]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [ ]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

In [ ]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)

    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
test_prediction(0, W1, b1, W2, b2)
test_prediction(1, W1, b1, W2, b2)
test_prediction(2, W1, b1, W2, b2)
test_prediction(3, W1, b1, W2, b2)

In [ ]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)